In [42]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import azureml.core
from azureml.core import Workspace, Datastore, Dataset, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.dnn import PyTorch
from azure.storage.blob import BlockBlobService
# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.65


In [12]:
ws = Workspace.from_config()

project_folder = './player_name'
os.makedirs(project_folder, exist_ok=True)

experiment_name = 'player_name'
experiment = Experiment(ws, name=experiment_name)

In [4]:
datastore = Datastore.register_azure_blob_container(workspace=ws, 
                                                      datastore_name='omnicintellige6390360867', 
                                                      container_name='playername',
                                                      account_name='omnicintellige6390360867', 
                                                      account_key='Z1u6QIYTp8QOP+ECSUcyCtNk+zBLBoq7jdsyvG64fbVsv/0MBzlHxFKNObVB3MVn9MZE2rlVQJXSv5TaBw4dHQ==')

In [5]:
ds = Datastore.get(ws, 'omnicintellige6390360867')

In [43]:
block_blob_service = BlockBlobService(account_name='omnicintellige6390360867', account_key='Z1u6QIYTp8QOP+ECSUcyCtNk+zBLBoq7jdsyvG64fbVsv/0MBzlHxFKNObVB3MVn9MZE2rlVQJXSv5TaBw4dHQ==')

container_name ='playername'
generator = block_blob_service.list_blobs(container_name)
datastore_paths = []
for blob in generator:
    datastore_paths.append((datastore, blob.name))

name_ds = Dataset.File.from_files(path=datastore_paths)

In [45]:
print(name_ds)
name_ds = name_ds.register(workspace = ws,
                                 name = 'player_name_ds',
                                 description = 'Test of player name OCR',
                                 create_new_version = True)

FileDataset
{
  "source": [
    "('omnicintellige6390360867', '7015.hdf5')",
    "('omnicintellige6390360867', '7016.hdf5')",
    "('omnicintellige6390360867', '7017.hdf5')",
    "('omnicintellige6390360867', '7018.hdf5')",
    "('omnicintellige6390360867', '7019.hdf5')",
    "('omnicintellige6390360867', '7022.hdf5')",
    "('omnicintellige6390360867', '7023.hdf5')",
    "('omnicintellige6390360867', '7024.hdf5')",
    "('omnicintellige6390360867', '7025.hdf5')",
    "('omnicintellige6390360867', '7026.hdf5')",
    "('omnicintellige6390360867', '7088.hdf5')",
    "('omnicintellige6390360867', '7089.hdf5')",
    "('omnicintellige6390360867', '7090.hdf5')",
    "('omnicintellige6390360867', '7091.hdf5')",
    "('omnicintellige6390360867', '7319.hdf5')",
    "('omnicintellige6390360867', '7326.hdf5')",
    "('omnicintellige6390360867', '7327.hdf5')",
    "('omnicintellige6390360867', '7328.hdf5')",
    "('omnicintellige6390360867', '7329.hdf5')",
    "('omnicintellige6390360867', '7330.h

In [44]:
name_ds.to_path()

array(['/omnicintellige6390360867/7015.hdf5',
       '/omnicintellige6390360867/7016.hdf5',
       '/omnicintellige6390360867/7017.hdf5',
       '/omnicintellige6390360867/7018.hdf5',
       '/omnicintellige6390360867/7019.hdf5',
       '/omnicintellige6390360867/7022.hdf5',
       '/omnicintellige6390360867/7023.hdf5',
       '/omnicintellige6390360867/7024.hdf5',
       '/omnicintellige6390360867/7025.hdf5',
       '/omnicintellige6390360867/7026.hdf5',
       '/omnicintellige6390360867/7088.hdf5',
       '/omnicintellige6390360867/7089.hdf5',
       '/omnicintellige6390360867/7090.hdf5',
       '/omnicintellige6390360867/7091.hdf5',
       '/omnicintellige6390360867/7319.hdf5',
       '/omnicintellige6390360867/7326.hdf5',
       '/omnicintellige6390360867/7327.hdf5',
       '/omnicintellige6390360867/7328.hdf5',
       '/omnicintellige6390360867/7329.hdf5',
       '/omnicintellige6390360867/7330.hdf5',
       '/omnicintellige6390360867/7331.hdf5',
       '/omnicintellige6390360867/

In [13]:
cluster_name = "gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Creating a new compute target...
Creating
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


In [46]:

script_params = {
    '--data_folder': name_ds.as_named_input('names').as_mount(),
    '--beta': 0.5,
    '--learning_rate': 0.001,
    '--num_epochs': 20,
    '--early_stopping': 2,
    '--output_dir': './outputs'
}

estimator = PyTorch(source_directory=project_folder, 
                    script_params=script_params,
                    compute_target=compute_target,
                    entry_script='pytorch_train.py',
                    use_gpu=True,
                    pip_packages=['h5py', 'azureml-sdk',
                                  'azureml-dataprep[pandas,fuse]>=1.1.14'])

WARNING - framework_version is not specified, defaulting to version 1.2.


In [47]:
run = experiment.submit(estimator)
run.wait_for_completion(show_output=True)

RunId: player_name_1570877392_fbf43a88
Web View: https://mlworkspace.azure.ai/portal/subscriptions/1d1428b3-3300-4696-99ad-a1df55934195/resourceGroups/omnic_intelligence/providers/Microsoft.MachineLearningServices/workspaces/omnic_intelligence/experiments/player_name/runs/player_name_1570877392_fbf43a88

Streaming azureml-logs/55_azureml-execution-tvmps_e235a838d141b51a45d8f35b3abcfaab48846b3eee3a5249ef611106447dfd3c_d.txt

2019-10-12T10:54:04Z Successfully mounted a/an Azure File Shares at /mnt/batch/tasks/shared/LS_root/jobs/omnic_intelligence/azureml/player_name_1570877392_fbf43a88/mounts/workspacefilestore
2019-10-12T10:54:04Z Mounted //omnicintellige6390360867.file.core.windows.net/azureml-filestore-6085e42c-59ef-4f3b-a949-7099435c60e9 at /mnt/batch/tasks/shared/LS_root/jobs/omnic_intelligence/azureml/player_name_1570877392_fbf43a88/mounts/workspacefilestore
2019-10-12T10:54:04Z No blob file systems configured
2019-10-12T10:54:04Z No unmanaged file systems configured
2019-10-12T10

WARNING - Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', OSError("(10060, 'WSAETIMEDOUT')"))': /azureml/ExperimentRun/dcid.player_name_1570877392_fbf43a88/azureml-logs/70_driver_log.txt?sv=2018-11-09&sr=b&sig=LhaTFt9THs2TBLMl%2BVmyK%2Fo9GiD%2FnvELCs5V4hwRl44%3D&st=2019-10-12T11%3A12%3A27Z&se=2019-10-12T19%3A22%3A27Z&sp=r


ChunkedEncodingError: ('Connection broken: OSError("(10054, \'WSAECONNRESET\')")', OSError("(10054, 'WSAECONNRESET')"))

In [36]:
model = run.register_model(model_name='player_name_ocr', model_path='outputs/')

In [ ]:
os.makedirs('./model', exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join('./model', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)

In [48]:
PyTorch.get_supported_versions()


['1.0', '1.1', '1.2']